api: https://pittsburgh.publicbikesystem.net/customer/gbfs/v2/gbfs.json

In [2]:
import csv
import requests
import pandas as pd
import time
import pyodbc
import sqlalchemy as sa
import urllib
import time
from datetime import datetime
from sqlalchemy import update

In [23]:
server = "database-1.czm6aegec3xq.us-east-2.rds.amazonaws.com"
database = "pogoh"
username = "master"
password = "egahIeae$aevnef#4"
driver = "{ODBC Driver 18 for SQL Server}"
connectionString = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes'
conn = pyodbc.connect(connectionString)

params = urllib.parse.quote_plus(
    'Driver=%s;' % driver +
    'Server=tcp:%s,1433;' % server +
    'Database=%s;' % database +
    'Uid=%s;' % username +
    'Pwd={%s};' % password +
    'Encrypt=yes;' +
    'TrustServerCertificate=yes;' +
    'Connection Timeout=90;')

conn_str = 'mssql+pyodbc:///?odbc_connect=' + params

engine = sa.create_engine(conn_str)

r = requests.get('https://pittsburgh.publicbikesystem.net/customer/gbfs/v2/en/station_status')
j = r.json()

stock = pd.DataFrame(j['data']['stations'])
stock['last_reported'] = pd.to_datetime(stock['last_reported'], unit = 's')
stock['global_update_time'] = pd.to_datetime(datetime.utcnow())
stock['id'] = stock['station_id'].apply(lambda x: str(x)) + '@' + stock['global_update_time'].apply(lambda x: str(x))
stock.set_index(['id'])

sql = ("SELECT * FROM fact_stock")
data = pd.read_sql(sql, conn)

stock[data.columns].to_sql("fact_stock", engine, index = False, if_exists = 'append', chunksize = 10)
conn.commit()
conn.close()
print('Finished refreshing stock')
stock.head()

Finished refreshing stock


,station_id,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,traffic,vehicle_docks_available,vehicle_types_available,global_update_time,id
0,1,7,2,10,0,2024-02-18 01:03:10,True,IN_SERVICE,True,True,True,None,"[{'vehicle_type_ids': ['FIT', 'EFIT', 'ICONIC'...","[{'vehicle_type_id': 'FIT', 'count': 3}, {'veh...",2024-02-18 01:04:32.616668,1@2024-02-18 01:04:32.616668
1,2,6,0,9,0,2024-02-18 01:00:57,False,IN_SERVICE,True,True,True,None,"[{'vehicle_type_ids': ['FIT', 'EFIT', 'ICONIC'...","[{'vehicle_type_id': 'FIT', 'count': 5}, {'veh...",2024-02-18 01:04:32.616668,2@2024-02-18 01:04:32.616668
2,3,4,2,9,0,2024-02-18 01:03:05,False,IN_SERVICE,True,True,True,None,"[{'vehicle_type_ids': ['FIT', 'EFIT', 'ICONIC'...","[{'vehicle_type_id': 'FIT', 'count': 1}, {'veh...",2024-02-18 01:04:32.616668,3@2024-02-18 01:04:32.616668
3,4,7,0,8,0,2024-02-18 01:03:00,False,IN_SERVICE,True,True,True,None,"[{'vehicle_type_ids': ['FIT', 'EFIT', 'ICONIC'...","[{'vehicle_type_id': 'FIT', 'count': 2}, {'veh...",2024-02-18 01:04:32.616668,4@2024-02-18 01:04:32.616668
4,5,7,1,11,0,2024-02-18 01:03:54,False,IN_SERVICE,True,True,True,None,"[{'vehicle_type_ids': ['FIT', 'EFIT', 'ICONIC'...","[{'vehicle_type_id': 'FIT', 'count': 3}, {'veh...",2024-02-18 01:04:32.616668,5@2024-02-18 01:04:32.616668


In [11]:
server = "database-1.czm6aegec3xq.us-east-2.rds.amazonaws.com"
database = "pogoh"
username = "master"
password = "egahIeae$aevnef#4"
driver = "{ODBC Driver 18 for SQL Server}"
connectionString = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes'
conn = pyodbc.connect(connectionString)

params = urllib.parse.quote_plus(
    'Driver=%s;' % driver +
    'Server=tcp:%s,1433;' % server +
    'Database=%s;' % database +
    'Uid=%s;' % username +
    'Pwd={%s};' % password +
    'Encrypt=yes;' +
    'TrustServerCertificate=yes;' +
    'Connection Timeout=90;')

conn_str = 'mssql+pyodbc:///?odbc_connect=' + params

engine = sa.create_engine(conn_str)


r = requests.get('https://pittsburgh.publicbikesystem.net/customer/gbfs/v2/en/vehicle_types')
j = r.json()
bike_types = pd.DataFrame(j['data']['vehicle_types'])
bike_types['bike_name'] = bike_types['name']
bike_types.set_index(['bike_name'])

sql = ("SELECT * FROM bike_types")
data = pd.read_sql(sql, conn)

conn.execute("DELETE FROM bike_types")
conn.commit()
print('bike_types cleared')

bike_types[data.columns].to_sql("bike_types", engine, if_exists = 'append', index = False, chunksize = 5)
conn.commit()
print('Finished refreshing bike_types')

bike_types cleared
Finished refreshing bike_types


In [12]:
server = "database-1.czm6aegec3xq.us-east-2.rds.amazonaws.com"
database = "pogoh"
username = "master"
password = "egahIeae$aevnef#4"
driver = "{ODBC Driver 18 for SQL Server}"
connectionString = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes'
conn = pyodbc.connect(connectionString)

params = urllib.parse.quote_plus(
    'Driver=%s;' % driver +
    'Server=tcp:%s,1433;' % server +
    'Database=%s;' % database +
    'Uid=%s;' % username +
    'Pwd={%s};' % password +
    'Encrypt=yes;' +
    'TrustServerCertificate=yes;' +
    'Connection Timeout=90;')

conn_str = 'mssql+pyodbc:///?odbc_connect=' + params

engine = sa.create_engine(conn_str)
r = requests.get('https://pittsburgh.publicbikesystem.net/customer/gbfs/v2/en/station_information')
j = r.json()
stations = pd.DataFrame(j['data']['stations'])
stations.set_index(['station_id'])
sql = ("SELECT * FROM stations")
data = pd.read_sql(sql, conn)
conn.execute("DELETE FROM stations")
conn.commit()
print('stations cleared')

stations[data.columns].to_sql("stations", engine, if_exists = 'append', index = False, chunksize = 5)
conn.commit()
print('Finished refreshing stations')

stations cleared
Finished refreshing stations
